# DeepBind using Keras - util.py

In Util sono presenti le funzioni che verranno utilizzate in tutti gli esprimenti.
Importiamo prima di tutto le librerie necessarie all'esecuzione del codice:

In [ ]:
import tensorflow as tf
import numpy as np
import math
import random
from keras import backend as K


Ora inizializziamo le variabili:

In [ ]:
dictpad={'A':[1.,0.,0.,0.],'C':[0.,1.,0.,0.],'G':[0.,0.,1.,0.],'T':[0.,0.,0.,1.],'U':[0.,0.,0.,1.],'N':[0.25,0.25,0.25,0.25]}
dictReverse={'A':'T','C':'G','G':'C','T':'A','N':'N'} #dictionary to implement reverse-complement mode

Seguendo il paper supplementare (pag.3), definiamo la funzione seqtopad, che trasforma la sequenza in un array.
Ho deciso in Keras di creare un array in una dimensione, piuttosto che 2-dimensionale.
Il motivo è che la convoluzione si sposta solo verticalmente, quindi la si può fare 1 dimensionale spostandosi di strides=4

In [ ]:
def seqtopad(sequence,motlen):    
    pad=[]
    for j in range(motlen-1):
        pad.extend(dictpad['N'])
    res=pad.copy()
    for i in range(len(sequence)):
        res.extend(dictpad[sequence[i]])
    res.extend(pad)
    return np.asarray(res)

Definiamo le funzioni padsequence, che permette di uniformare la lunghezza di esperimenti con sequenze di diversa lunghezza aggiungendo delle 'N' (non-basi).

In [ ]:
def padsequence(sequence,maxlength):
    return sequence + 'N'*(maxlength-len(sequence))

def reverse(sequence):
    revseq=''
    for i in sequence:
        revseq+=dictReverse[i]
    return revseq   

Definiamo le funzioni logsampler e sqrtsampler, necessario per generare un numero casuale tramite queste distribuzioni (paper supp. pag.12).

In [ ]:
def logsampler(a,b,tensor=0):
    if(tensor==1):
        x=tf.Variable(tf.random_uniform([1],minval=0,maxval=1))
    else:
        x=np.random.uniform(low=0,high=1)
    y=10**((math.log10(b)-math.log10(a))*x + math.log10(a))
    return y

def sqrtsampler(a,b,tensor=0):
    if(tensor==1):
        x=tf.Variable(tf.random_uniform([1],minval=0,maxval=1))
    else:
        x=np.random.uniform(low=0,high=1)
    y=(b-a)*math.sqrt(x)+a
    return y

Per gli esperimenti SELEX e CHIP-seq la funzione di perdita stabilita dagli autori è la negative-log-likelihood (non presente nelle losses di default di Keras).
Definiamo perciò questa funzione e una funzioni che implementi il dinucleotide-shuffle, necessaria per costruire le entries negative (specificità zero) nel training set di SELEX e CHIP-seq.

In [ ]:
def log_loss(label,prediction):
    def sigma(x):
        return 1/(1+math.e**(-x))
    return K.mean(-label*K.log(sigma(prediction)) - (1-label)*K.log(1-sigma(prediction)))
    
def dinucshuffle(sequence):
    b=[sequence[i:i+2] for i in range(0, len(sequence), 2)]
    random.shuffle(b)
    d=''.join([str(x) for x in b])
    return d